In [3]:
import open3d as o3d
mesh = o3d.io.read_triangle_mesh("room_mesh_closed.ply")
pcd = mesh.sample_points_poisson_disk(number_of_points=100000, init_factor=5)#.sample_points_uniformly(number_of_points=100000)
o3d.visualization.draw_geometries([pcd])

In [4]:
o3d.io.write_point_cloud("room_point_cloud_closed.ply", pcd)

True

In [12]:
"""
Please cite the following paper (related to this topic), if this code helps you in your research.
   @article{xia2020geometric,
            title={Geometric primitives in LiDAR point clouds: A review},
            author={Xia, Shaobo and Chen, Dong and Wang, Ruisheng and Li, Jonathan and Zhang, Xinchang},
            journal={IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing},
            volume={13},
            pages={685--707},
            year={2020},
            publisher={IEEE}
           }
    Parts of codes are from https://github.com/CGAL/cgal-swig-bindings/blob/main/examples/python/Shape_detection_example.py
    and from https://github.com/GeoVectorMatrix/Open3D_Based_Py/blob/main/Algorithms/CGALShapes/CGALShapesO3D_Demo.py
"""      
from CGAL.CGAL_Kernel import Point_3
from CGAL.CGAL_Kernel import Vector_3
from CGAL.CGAL_Point_set_3 import Point_set_3
from CGAL.CGAL_Shape_detection import *
import open3d as o3d
import os
import numpy as np
import matplotlib.pyplot as plt

datafile = "room_point_cloud_closed.ply"
points = Point_set_3(datafile)

# Prepare data for open3d
npPt = np.zeros((points.size(), 3))
npNor = np.zeros((points.size(), 3))
npCol =  np.zeros((points.size(), 3))
for i in range(points.size()):
    ar = np.array([points.point(i).x(),points.point(i).y(),points.point(i).z()])
    nor = np.array([points.normal_map().get(i).x(), points.normal_map().get(i).y(), points.normal_map().get(i).z()])
    col = np.array([points.int_map("red").get(i), points.int_map("green").get(i), points.int_map("blue").get(i)]) / 255
    npPt[i] = ar.copy()
    npNor[i] = nor.copy()   
    npCol[i] = col.copy()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(npPt)
pcd.normals = o3d.utility.Vector3dVector(npNor)
pcd.colors = o3d.utility.Vector3dVector(npCol)

o3d.visualization.draw_geometries([pcd])  # show original point clouds.


In [6]:

# CGAL plane detection by the region growing
print(points.size(), "points read")
print("Detecting planes with region growing (sphere query)")
plane_map = points.add_int_map("plane_index")
nb_planes = region_growing(points, plane_map, min_points=1000)

print(nb_planes, "planes(s) detected")
#

100000 points read
Detecting planes with region growing (sphere query)
13 planes(s) detected


In [7]:
labels  = np.zeros(points.size()).astype(int)
PlaneList = [[] for i in range(nb_planes+1)]  # index -1 for others
for i in range(points.size()):
    labels[i] = points.int_map('plane_index').get(i)
    PlaneList[labels[i]+1].append(i)
print(len(PlaneList))
# Visualizer
max_label = len(PlaneList)
print(f"point cloud has {max_label} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 1  # set to white for small clusters (label - 0 )
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd])

14
point cloud has 14 clusters


In [8]:
labels  = np.zeros(points.size()).astype(int)
PlaneList = [[] for i in range(nb_planes+1)]  # index -1 for others
PlanePointDict = {}
NormalPointDict = {}
PlaneColDict = {}
for i in range(points.size()):
    pl_idx = points.int_map('plane_index').get(i)
    labels[i] = pl_idx
    PlaneList[labels[i]+1].append(i)
    if pl_idx not in PlanePointDict.keys():
        PlanePointDict[pl_idx] = []
    if pl_idx not in PlaneColDict.keys():
        PlaneColDict[pl_idx] = []
    if pl_idx not in NormalPointDict.keys():
        NormalPointDict[pl_idx] = []
    ar = np.array([points.point(i).x(),points.point(i).y(),points.point(i).z()])
    nor = np.array([points.normal_map().get(i).x(), points.normal_map().get(i).y(), points.normal_map().get(i).z()])
    col = np.array([points.int_map("red").get(i), points.int_map("green").get(i), points.int_map("blue").get(i)]) / 255
    
    PlanePointDict[pl_idx].append(ar)  
    NormalPointDict[pl_idx].append(nor)   
    PlaneColDict[pl_idx].append(col)

print(len(PlaneList))

14


In [9]:
from sklearn.cluster import DBSCAN
import numpy as np
import sklearn


geos = []

for j in PlanePointDict.keys():

    npPt = np.zeros((len(PlanePointDict[j]), 3))
    npNor =  np.zeros((len(NormalPointDict[j]), 3))
    npCol =  np.zeros((len(PlaneColDict[j]), 3))
    
    npDb = np.zeros((len(PlaneColDict[j]), 6))
    for i in range(len(PlanePointDict[j])):
        npPt[i] = PlanePointDict[j][i].copy()
        npNor[i] = NormalPointDict[j][i].copy()
        npCol[i] = PlaneColDict[j][i].copy()
        
        npDb[i] = np.concatenate((npPt[i], npCol[i] * 25), axis=None)
    
    clustering = DBSCAN(eps=0.85).fit(npDb)
    
    max_label = len(set(clustering.labels_))
    print(f"point cloud has {max_label} clusters")
    colors = plt.get_cmap("tab20")(clustering.labels_ / (max_label if max_label > 0 else 1))
    colors[clustering.labels_ < 0] = 1  # set to white for small clusters (label - 0 )
    
    pcd_ = o3d.geometry.PointCloud()
    pcd_.points = o3d.utility.Vector3dVector(npPt)
    pcd_.normals = o3d.utility.Vector3dVector(npNor)
    pcd_.colors = o3d.utility.Vector3dVector(colors[:, :3])
                                            
    geos.append(pcd_)
    geos.append(pcd_.get_axis_aligned_bounding_box())

o3d.visualization.draw_geometries(geos)  # show original point clouds.

point cloud has 6 clusters
point cloud has 132 clusters
point cloud has 8 clusters
point cloud has 13 clusters
point cloud has 13 clusters
point cloud has 4 clusters
point cloud has 8 clusters
point cloud has 6 clusters
point cloud has 4 clusters
point cloud has 4 clusters
point cloud has 4 clusters
point cloud has 2 clusters
point cloud has 2 clusters
point cloud has 6 clusters


In [10]:
from sklearn.cluster import DBSCAN
import numpy as np
import sklearn


geos = []
for j in PlanePointDict.keys():
    if j != 3:
        continue
    npPt = np.zeros((len(PlanePointDict[j]), 3))
    npNor =  np.zeros((len(NormalPointDict[j]), 3))
    npCol =  np.zeros((len(PlaneColDict[j]), 3))
    
    npDb = np.zeros((len(PlaneColDict[j]), 6))
    for i in range(len(PlanePointDict[j])):
        npPt[i] = PlanePointDict[j][i].copy()
        npNor[i] = NormalPointDict[j][i].copy()
        npCol[i] = PlaneColDict[j][i].copy()
        
        npDb[i] = np.concatenate((npPt[i], npCol[i] * 25), axis=None)
    
    clustering = DBSCAN(eps=0.85).fit(npDb)
    
    max_label = len(set(clustering.labels_))
    print(f"point cloud has {max_label} clusters")
    colors = plt.get_cmap("tab20")(clustering.labels_ / (max_label if max_label > 0 else 1))
    colors[clustering.labels_ < 0] = 1  # set to white for small clusters (label - 0 )
    
    pcd_ = o3d.geometry.PointCloud()

    pcd_.points = o3d.utility.Vector3dVector(npPt)
    pcd_.normals = o3d.utility.Vector3dVector(npNor)
    pcd_.colors = o3d.utility.Vector3dVector(colors[:, :3])
                                            
    geos.append(pcd_)
    geos.append(pcd_.get_axis_aligned_bounding_box())



o3d.visualization.draw_geometries(geos)  # show original point clouds.

point cloud has 13 clusters


In [11]:
o3d.io.write_point_cloud("room_point_cloud_closed_building_element.pts", pcd_)

True